In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
-- Welcome to Snowflake Notebooks!
-- Try out a SQL cell to generate some data.
SELECT 'FRIDAY' as SNOWDAY, 0.2 as CHANCE_OF_SNOW
UNION ALL
SELECT 'SATURDAY',0.5
UNION ALL 
SELECT 'SUNDAY', 0.9;

In [ ]:
# Then, we can use the python name to turn cell2 into a Pandas dataframe
my_df = cell2.to_pandas()

# Chart the data
st.subheader("Chance of SNOW ❄️")
st.line_chart(my_df, x='SNOWDAY', y='CHANCE_OF_SNOW')

# Give it a go!
st.subheader("Try it out yourself and show off your skills 🥇")

In [ ]:
USE ROLE SYSADMIN;
CREATE SCHEMA BLAZEJ_DEV.FORECAST_EXAMPLE;

In [ ]:
CREATE OR REPLACE FILE FORMAT BLAZEJ_DEV.FORECAST_EXAMPLE.CSV_FF
    type = 'csv'
    SKIP_HEADER = 1,
    COMPRESSION = AUTO;


CREATE OR REPLACE STAGE BLAZEJ_DEV.FORECAST_EXAMPLE.LOAD_STAGE
    url = 's3://sfquickstarts/frostbyte_tastybytes/mlpf_quickstart/'
    file_format = BLAZEJ_DEV.FORECAST_EXAMPLE.CSV_FF;

In [ ]:
CREATE OR REPLACE TABLE BLAZEJ_DEV.FORECAST_EXAMPLE.TASTY_BYTES_SALES(
  	DATE DATE,
	PRIMARY_CITY VARCHAR(16777216),
	MENU_ITEM_NAME VARCHAR(16777216),
	TOTAL_SOLD NUMBER(17,0)
);

In [ ]:
COPY INTO BLAZEJ_DEV.FORECAST_EXAMPLE.TASTY_BYTES_SALES
    FROM @BLAZEJ_DEV.FORECAST_EXAMPLE.LOAD_STAGE/ml_functions_quickstart.csv;

SELECT * FROM BLAZEJ_DEV.FORECAST_EXAMPLE.TASTY_BYTES_SALES LIMIT 10;

In [ ]:
CREATE OR REPLACE TABLE BLAZEJ_DEV.FORECAST_EXAMPLE.VANCOUVER_SALES AS (
    SELECT
        to_timestamp_ntz(date) AS timestamp,
        primary_city,
        menu_item_name,
        total_sold
    FROM
        BLAZEJ_DEV.FORECAST_EXAMPLE.TASTY_BYTES_SALES
    WHERE
        date > (SELECT max(date) - interval '1 year' FROM BLAZEJ_DEV.FORECAST_EXAMPLE.TASTY_BYTES_SALES)
    GROUP BY ALL
);

SELECT * FROM BLAZEJ_DEV.FORECAST_EXAMPLE.VANCOUVER_SALES;

In [ ]:
CREATE OR REPLACE VIEW BLAZEJ_DEV.FORECAST_EXAMPLE.LOBSTER_SALES AS (
    SELECT
        timestamp,
        total_sold
    FROM
        BLAZEJ_DEV.FORECAST_EXAMPLE.VANCOUVER_SALES 
    WHERE
        menu_item_name LIKE 'Lobster Mac & Cheese'
);

In [ ]:
// https://docs.snowflake.com/en/sql-reference/functions/system_reference
// https://docs.snowflake.com/en/developer-guide/stored-procedure/stored-procedures-calling-references#label-reference-object-example

// https://docs.snowflake.com/en/sql-reference/classes/forecast
CREATE OR REPLACE SNOWFLAKE.ML.FORECAST BLAZEJ_DEV.FORECAST_EXAMPLE.LOBSTER_FORECAST (
    INPUT_DATA => SYSTEM$REFERENCE('VIEW', 'BLAZEJ_DEV.FORECAST_EXAMPLE.LOBSTER_SALES'),
    TIMESTAMP_COLNAME => 'TIMESTAMP',
    TARGET_COLNAME => 'TOTAL_SOLD'
);

In [ ]:
SHOW SNOWFLAKE.ML.FORECAST;

In [ ]:
CALL BLAZEJ_DEV.FORECAST_EXAMPLE.LOBSTER_FORECAST!FORECAST(FORECASTING_PERIODS => 10)

In [ ]:
// UWAGA!! Ta komenda może nie działać poprawnie, query id najlepiej wyciagnac z query history (tylko w przypadku Notebook'ów)!
SELECT * FROM TABLE(RESULT_SCAN('01b99f98-0004-7475-0003-1b6300056e02'))

In [ ]:
// Zapiszmy prognozy do tabeli za pomocą funkcji umożliwiającej odpytywanie wyników przeszłych kwerend
// https://docs.snowflake.com/en/sql-reference/functions/result_scan
CREATE OR REPLACE TABLE BLAZEJ_DEV.FORECAST_EXAMPLE.LOBSTER_FORECAST AS (
    SELECT * FROM TABLE(RESULT_SCAN('01b99f98-0004-7475-0003-1b6300056e02'))
);

SELECT * FROM BLAZEJ_DEV.FORECAST_EXAMPLE.LOBSTER_FORECAST;

In [ ]:
// W notebook, wyswietla sie jedynie wynik końcowy, w worksheet możemy podejrzeć wykres
SELECT
    timestamp,
    total_sold,
    NULL AS forecast
FROM
    BLAZEJ_DEV.FORECAST_EXAMPLE.LOBSTER_SALES
WHERE
    timestamp > '2023-03-01'
UNION
SELECT
    TS AS timestamp,
    NULL AS total_sold,
    forecast
FROM
    BLAZEJ_DEV.FORECAST_EXAMPLE.LOBSTER_FORECAST
ORDER BY
    timestamp asc;

In [ ]:
// Prosta zamiana kwerendy aby pokazać ją za pomocą metod Streamlit'owych

SELECT
    timestamp,
    total_sold,
    FALSE AS is_forecast
FROM
    BLAZEJ_DEV.FORECAST_EXAMPLE.LOBSTER_SALES
WHERE
    timestamp > '2023-03-01'
UNION
SELECT
    TS AS timestamp,
    forecast AS total_sold,
    TRUE AS is_forecast
FROM
    BLAZEJ_DEV.FORECAST_EXAMPLE.LOBSTER_FORECAST
ORDER BY
    timestamp asc;

In [ ]:
forecast_df = cell16.to_pandas()

st.subheader("Prognoza sprzedaży dla Homarów z Mac'n Cheese")
st.line_chart(forecast_df, x='TIMESTAMP', y='TOTAL_SOLD', color='IS_FORECAST')
